In [531]:

'''Clean up recipes gathered from the web, and normalize it into a new json database.'
'''

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
import re
import sys
import pprint
import json
import csv

import itertools
from collections import Counter

#Function to split the String.
def split_ingr(x):
    #take the long string in 'ingredients', lemmatize, regex, and split into words
    cleanlist=[]
    lst = x.strip().lower()
    test= lst.split(' ')
    clean = remove_word(test)
    return clean

#Remove stopwords
def remove_word(cleanlist):
    
    #Function that removes stopwords that are in alist and then return a string
    alist =['soft serve','rich', 'light','low fat', 'reduced fat','large','medium','small','dip','regular', ',','fat free', 'fatfree', 'nonfat','gluten free', 'free range',\
            'reduced sodium','serve','no','diet','frozen','meal','added','soft','or','part skim','meat free''low sugar', 'low','part','skim','eat','canned syrup',\
            'raw','lowfat' ,'lean', 'only','(eat lean & fat)','low fat','(75% lean, regular)','(95% lean)','low','(2%)','(skim)','(part skim)','(regular)' ,'fat','fat-free',\
            'sugar-free','soft serve','cooked','salt free','sodium free', 'low sodium', 'sweetened','unsweetened','large','extra large','oz ','chocolate', 'covered']
    
    for item in alist:
        if item in cleanlist:
            cleanlist.remove(item)
    return' '.join(cleanlist)

    
if __name__ == '__main__':
    
    #Open and read the csv file. Converts it into a pandas dataframe for data manipulation
    yum = pd.read_csv('Food_Display_Table - Sheet1.csv')
    yum.reset_index(inplace = True)
    
    #Dropping Collumns that are irrelevant
    yum.drop(['Food_Code','Portion_Default','Portion_Amount','Portion_Display_Name','Factor','Increment','Multiplier','Grains','Whole_Grains','Milk','Meats','Soy','Drybeans_Peas','Oils','Solid_Fats','Added_Sugars','Alcohol','Saturated_Fats'], axis = 1, inplace = True)
    yum.drop(['Vegetables','Orange_Vegetables','Drkgreen_Vegetables','Starchy_vegetables','Other_Vegetables','Fruits'], axis = 1, inplace = True)
    yum.drop(['index'], axis = 1, inplace = True)
    
    #renaming the display name collumn into cusine and than dropping the original collumn
    yum['cuisine'] = yum['Display_Name']
    yum.drop(['Display_Name'], axis = 1, inplace = True)
    
    #Adding missing data to the dataframe
    listOfSeries = [pd.Series([325.0, 'curry powder'], index=yum.columns ) ,
                pd.Series([0.0, 'salt'], index=yum.columns ) ,
                pd.Series([0.0, 'soda'], index=yum.columns ) ,
                pd.Series([117.0, 'oats'], index=yum.columns ) ,
                pd.Series([53.0, 'baking powder'], index=yum.columns ) ,
                pd.Series([247.4, 'cinnamon'], index=yum.columns ) ,
                pd.Series([364.0, 'flour'], index=yum.columns ) ,
                pd.Series([387.7, 'sugar'], index=yum.columns ) ,
                pd.Series([884.1, 'shortening'], index=yum.columns ) ,
                pd.Series([0.0, 'cup water'], index=yum.columns ) ,   
                pd.Series([0.0, 'hot water'], index=yum.columns ) ,   
                pd.Series([364.0, 'plain flour'], index=yum.columns ) ,
                pd.Series([716.0, 'melted butter'], index=yum.columns ) , 
                pd.Series([34.0, 'pizza sauce'], index=yum.columns ) ,
                pd.Series([20.0, 'apples chopped into 1.4 inch pieces'], index=yum.columns ) ,
                pd.Series([0.0, 'kosher salt'], index=yum.columns ) ,  
                pd.Series([884.1, 'olive oil'], index=yum.columns ) ,    
                pd.Series([884.1, 'vegetable oil'], index=yum.columns ) ,   
                pd.Series([335.1, 'cloves'], index=yum.columns ) , 
                pd.Series([364.0, 'all-purpose flour'], index=yum.columns ) ,
                pd.Series([117.0, 'rolled oats, (not instant)'], index=yum.columns ) , 
                pd.Series([380.0, 'packed brown sugar'], index=yum.columns ) ,  
                pd.Series([381.0, 'cornstarch'], index=yum.columns ) , 
                pd.Series([104.0, 'yeast'], index=yum.columns ) ]
    
    modDfObj = yum.append(listOfSeries , ignore_index=True)
   
    
    #replacing string values that are not in the american alphabet
    newlist=[]
    test = modDfObj['cuisine'].replace('[^a-zA-Z\s]+','', regex=True)
    
    #For loop that call the function split_ingr. The function splits and cleans up the dataset.returning a new list with semi-cleaned dataset
    for line in test:
        newtest =split_ingr(str(line))
        newlist.append(split_ingr(str(line)))
        
    
    #Functions for renaming rows and correcting spelling mistakes in the newlist
    for item in filter(lambda x: 'parmesan romano cheese' in x, newlist):
        newlist = [w.replace(item, 'parmesan cheese') for w in newlist]
        
    for item in filter(lambda x: 'cheese cheddar cheese' in x, newlist):
        newlist = [w.replace(item, 'cheddar cheese') for w in newlist]
        
    for item in filter(lambda x: 'cheddar swiss' in x, newlist):
        newlist = [w.replace(item, 'cheddar cheese') for w in newlist]
    
    for item in filter(lambda x: 'turkey breast lunch meat free' in x, newlist):
        newlist = [w.replace(item, 'turkey breast') for w in newlist]
        
    for item in filter(lambda x: 'italian dressing oil  vinegar' in x, newlist):
        newlist = [w.replace(item, 'dried Italian salad dressing mix') for w in newlist]
       
    for item in filter(lambda x: 'baked winter squash' in x, newlist):
        newlist = [w.replace(item, 'acorn squash') for w in newlist]
    
    for item in filter(lambda x: 'cranberry juice  juice' in x, newlist):
        newlist = [w.replace(item, 'dried cranberry') for w in newlist]
        
    for item in filter(lambda x: 'soy nuts' in x, newlist):
        newlist = [w.replace(item, 'chopped nuts') for w in newlist]
        
    for item in filter(lambda x: 'whole wheat english muffin' in x, newlist):
        newlist = [w.replace(item, 'english muffins') for w in newlist]
        
    for item in filter(lambda x: 'hard boiled egg' in x, newlist):
        newlist = [w.replace(item, 'eggs') for w in newlist]
    
    for item in filter(lambda x: 'beef broth consomme' in x, newlist):
        newlist = [w.replace(item, 'beef broth') for w in newlist]
        
    for item in filter(lambda x: 'lentils' in x, newlist):
        newlist = [w.replace(item, 'dried green lentils') for w in newlist]
        
    for item in filter(lambda x: 'white rice' in x, newlist):
        newlist = [w.replace(item, 'basmati rice') for w in newlist]
        
   
    #Converting the object list 'newlist' to the name collumn
    modDfObj['name'] = newlist
    
    #Rcreating the duplicate for the calories collumn
    modDfObj['calories'] = modDfObj['Calories']
    
    #Dropping the original calories collumn
    modDfObj.drop(['Calories'], axis = 1, inplace = True)
     
    #removing duplicates
    modDfObj.drop_duplicates(subset ="name", keep = 'first', inplace = True) 
    modDfObj.drop(['cuisine'], axis = 1, inplace = True)

    #Creating dicts within a dict, so simulates a json file
    Dict = {}
    Dict["ingredients"] = {}
    data_dict = modDfObj.to_dict(orient='records')
    Dict["ingredients"] = data_dict
    
    #prettify printing, no nessecarry line of code.
    indent=0
    print (json.dumps(Dict, indent=4))
    
    
    with open('food_data.json', 'w') as json_file:  
        json.dump(Dict, json_file)


{
    "ingredients": [
        {
            "name": "sour cream",
            "calories": 133.65
        },
        {
            "name": "ice cream",
            "calories": 267.33
        },
        {
            "name": "ice cream bar",
            "calories": 169.12
        },
        {
            "name": "ice cream sandwich",
            "calories": 143.37
        },
        {
            "name": "fudgesicle",
            "calories": 121.91
        },
        {
            "name": "sherbet",
            "calories": 277.92
        },
        {
            "name": "sugar ice milk",
            "calories": 152.64
        },
        {
            "name": "cheddar cheese",
            "calories": 113.8475
        },
        {
            "name": "feta goat cheese",
            "calories": 99.0
        },
        {
            "name": "mozzarella cheese",
            "calories": 85.617
        },
        {
            "name": "cottage cheese creamed",
            "calories": 108.15
  